# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data
### Question 1: 
#### Give me the artist, song title and song's length in the music app history that was heard during sessionId=338, and itemInSession=4
To answer this question, create table with specific columns is needed to store song by (session id and item in session) which is the primary keys. 
##### Column names will be:
##### session_id
##### itemInSession
##### artist_name
##### song_title
##### song_length
##### Then apply query: 
##### SELECT artist name, song title, length of the track FROM TABLE_NAME WHERE session_id = value AND itemInSession = value

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

##Create table
query = "CREATE TABLE IF NOT EXISTS song_session"
query = query + "(session_id INT, itemInSession INT, artist_name TEXT, song_title TEXT, song_length FLOAT, PRIMARY KEY(session_id, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# CSV file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_session (session_id, itemInSession, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
Q1_select = "SELECT artist_name, song_title, song_length FROM song_session WHERE session_id = %s and itemInSession = %s"
try:
    rows = session.execute(Q1_select, (338, 4))
except Exception as e:
    print(e)
    
table = PrettyTable(['Artist Name', 'Song Title', 'Song Length'])

for row in rows:
    table.add_row([row.artist_name, row.song_title, row.song_length])    

print(table)

+-------------+---------------------------------+--------------------+
| Artist Name |            Song Title           |    Song Length     |
+-------------+---------------------------------+--------------------+
|  Faithless  | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-------------+---------------------------------+--------------------+


### Question 2: 

#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer this question, create table with specific columns is needed to store data by (session id and user id) which is the primary keys. Item in session should be added to the primary key in order to sort the result and user id and session id is partition keys in order to make sure that sessions related to the same user on the same node because of big data. 

##### Column names will be:
##### user_id
##### session_id
##### itemInSession
##### artist_name
##### song_title
##### user_fn
##### user_ln
##### Then apply query: 
##### SELECT artist name, song title, first name, last name of user FROM TABLE_NAME WHERE session_id = value AND user_id = value


In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS artist_song_data "
query = query + "(user_id int, session_id int, itemInSession int, artist_name text, song_title text, user_fn text, user_ln text, PRIMARY KEY((user_id, session_id), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)             

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_data (user_id, session_id, itemInSession, artist_name, song_title, user_fn, user_ln)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query,(int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
##Add in the SELECT statement to verify the data was entered into the table
Q2_select = "SELECT artist_name, song_title, user_fn, user_ln FROM artist_song_data WHERE user_id = %s AND session_id = %s"
try:
    rows = session.execute(Q2_select, (10, 182))
except Exception as e:
    print(e)
    
table = PrettyTable(['Artist Name', 'Song Title', 'First Name', 'Last Name'])

for row in rows:
    table.add_row([row.artist_name, row.song_title, row.user_fn, row.user_ln])    

print(table)

+-------------------+------------------------------------------------------+------------+-----------+
|    Artist Name    |                      Song Title                      | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### Question 3: 
#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
To answer this question, create table with specific columns is needed to show who listenend to a specific song by (title of songs and user id) which is the primary keys. 

##### Column names will be:
##### user_id
##### song_title
##### user_fn
##### user_ln
##### Then apply query: 
##### SELECT user ID, song title, first name, last name of user FROM TABLE_NAME WHERE song_title = value AND user_id = value


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


query = "CREATE TABLE IF NOT EXISTS user_song "
query = query + "(user_id int, song_title text, user_fn text, user_ln text, PRIMARY KEY((song_title), user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)             
                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song (user_id, song_title, user_fn, user_ln)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query,( int(line[10]),line[9], line[1], line[4]))

In [20]:
## Add in the SELECT statement to verify the data was entered into the table
Q3_select = "SELECT user_fn, user_ln, song_title FROM user_song WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(Q3_select)
except Exception as e:
    print(e)
    
table = PrettyTable(['Song Title', 'First Name', 'Last Name'])

for row in rows:
    table.add_row([row.user_fn, row.user_ln, row.song_title])    

print(table)

+------------+------------+---------------------------+
| Song Title | First Name |         Last Name         |
+------------+------------+---------------------------+
| Jacqueline |   Lynch    | All Hands Against His Own |
|   Tegan    |   Levine   | All Hands Against His Own |
|    Sara    |  Johnson   | All Hands Against His Own |
+------------+------------+---------------------------+


### Drop the tables before closing out the sessions

In [25]:
##Drop the table before closing out the sessions
drop_song_session =  "DROP TABLE song_session"
try:
    rows = session.execute(drop_song_session)
except Exception as e:
    print(e)

drop_artist_song_data = "DROP TABLE artist_song_data"
try:
    rows = session.execute(drop_artist_song_data)
except Exception as e:
    print(e)

drop_user_song = "DROP TABLE user_song"
try:
    rows = session.execute(drop_user_song)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()